In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

import pprint
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

In [ ]:
#kaggleのkernelで実行する場合はファイルは../input/~にある
df_result = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/example_test_files/sample_submission.csv')
df=pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv')
df.head()

In [ ]:
df.nunique()


In [ ]:
df.isnull().sum()

In [ ]:
#欠損値を含む行削除 欠損値を無視して計算df.sum(skipna=False)
df.dropna(axis=1,inplace=True)

In [ ]:
#時系列データは処理できない
df.describe(include='all')#include='O'で上４行出力

In [ ]:
#スケール違う用に標準化
df_std = StandardScaler()
#特徴量リスト取得
columns = df.columns.values 
to_df_std = pd.DataFrame(df_std.fit_transform(df.loc[:,[columns]]))

#標準化処理したカラムをもとのカラムに反映
for column in columns:
    df[column] = to_df_std[column]

In [ ]:
df.reset_index(drop=True,inplace=True).set_index('reindex')

In [ ]:
#分布見てから判断
#DateカラムはRankに関係しなそうなため削除





In [ ]:
#データ分布

#したコード　ヒートマップで十分
# columns2 = df.columns.values 
# for i in columns:
#     for j in columns:
#         if j == i:
#             continue
    
    
#     #相関係数
#     corr[i][j]=plt.corr(i,j)
#     subplot()
#     #散布図
#     plt.scatter(i,j)


In [ ]:
#いらないカラム削除
x =
y =

In [ ]:
#データ分割(ホールドアウト法)

# x = pd.drop(df['Date'])

In [ ]:
#予測はアンサンブル予定
#時系列データ分析は初めてのため
#optunaを使いたい
#%matplotlib inline=output行にグラフ出力　plt.show()しなくてもグラフ表示

In [ ]:
#交差検証法
skf = StratifiedKFold(n_splits=10,
                      shuffle=True,
                      random_state=0)

In [ ]:
rf = RandomForestClassifier(random_state = 43,
                               n_jobs = -1,
                               oob_score=True)
# パラメーターを設定する
param_grid = {"n_estimators":[100,500,1000], # 2000まではいらない
              "max_features": [1, 2, 3, 4, 5, 7, 10],
              "max_depth": [3,5,7,10,15,None], #,20,30は過学習を引き起こす
              "min_samples_leaf":  [1, 2, 4],
              "min_samples_split": [2, 5, 10]
             } 
# パラメータチューニングをグリッドサーチ
rf_grid = GridSearchCV(estimator = rf,
                           param_grid = param_grid,
                           scoring = 'balanced_accuracy', #accuracy, balanced_accuracy
                           cv = skf,
                           return_train_score = True,
                           n_jobs = -1)
rf_grid.fit(data_set, target_set)

In [ ]:
#？分類器とパラメータの違いとは
(GBDT+RFで以下3つができる)
pprint.pprint(grid_result.best_estimator_)
pprint.pprint(grid_result.best_params_)
pprint.pprint(grid_result.best_score_)

In [ ]:
lgbmodel = lgb.LGBMClassifier(silent=False)
#pprint.pprint(model.get_params())
# パラメーターを設定する
param_grid = {"max_depth": [10, 25, 50, 75],
              "learning_rate" : [0.001,0.01,0.05,0.1],
              "num_leaves": [100,300,900,1200],
              "n_estimators": [100,200,500]
             }
# パラメータチューニングをグリッドサーチで行うために設定する
## このGridSearchCV には注意が必要 scoring は そのスコアを基準にして最適化する
lgb_grid = GridSearchCV(estimator = lgbm,
                           param_grid = param_grid,
                           scoring = 'balanced_accuracy',
                           cv = skf,
                           verbose=3,
                           return_train_score = True,
                           n_jobs = -1)

lgbm_grid.fit(data_set, target_set)

In [ ]:
#？分類器とパラメータの違いとは

# GBDT+RFで以下3つができる
pprint.pprint(grid_result.best_estimator_)
pprint.pprint(grid_result.best_params_)
pprint.pprint(grid_result.best_score_)

In [ ]:
#xgboost
xgb_model = xgb.XGBClassifier()
#pprint.pprint(model.get_params())
# パラメーターを設定する
param_grid = {"max_depth": [ 3, 6, 10,25], #10, 25,
              "learning_rate" : [0.0001,0.001,0.01], # 0.05,0.1
              "min_child_weight" : [1,3,6],
              "n_estimators": [100,200,300], # 500
              "subsample": [0.5,0.75,0.9],
              "gamma":[0,0.1,0.2],
              "eta": [0.3,0.15,0.10]
             }
# パラメータチューニングをグリッドサーチで行うために設定する
## このGridSearchCV には注意が必要 scoring は そのスコアを基準にして最適化する
xgb_grid = GridSearchCV(estimator = xgb_model,
                           param_grid = param_grid,
                           scoring = 'balanced_accuracy',
                           cv = skf,
                           verbose=3,
                           return_train_score = True,
                           n_jobs = -1)
xgb_grid.fit(data_set, target_set)

In [ ]:
#？分類器とパラメータの違いとは
(GBDT+RFで以下3つができる)
pprint.pprint(grid_result.best_estimator_)
pprint.pprint(grid_result.best_params_)
pprint.pprint(grid_result.best_score_)

In [ ]:
#catboost
catb = CatBoostClassifier()
#pprint.pprint(model.get_params())
# パラメーターを設定する
param_grid = {'depth': [4, 7, 10],
         'learning_rate' : [0.01, 0.1, 0.15],
         'l2_leaf_reg': [1,4,9],
         'iterations': [300]}
# パラメータチューニングをグリッドサーチで行うために設定する
## このGridSearchCV には注意が必要 scoring は そのスコアを基準にして最適化する
catb_grid = GridSearchCV(estimator = catb,
                           param_grid = param_grid,
                           scoring = 'accuracy',
                           cv = skf,
                           verbose=3,
                           return_train_score = True,
                           n_jobs = -1)
catb_grid.fit(data_set, target_set)

In [ ]:
#？分類器とパラメータの違いとは
(GBDT+RFで以下3つができる)
pprint.pprint(grid_result.best_estimator_)
pprint.pprint(grid_result.best_params_)
pprint.pprint(grid_result.best_score_)

In [ ]:
def get_max(*bin):
    return np.max(bin)

In [ ]:
get_max(rf_grid.predict(target_set),lgb_grid.predict(target_set),xgb_grud.predict(target_set),catb_grid.predict(target_set))

In [ ]:
to_csv_submission('jsx3.csv')